# Задание

Реализация upsampling блока нейронной сети UNet.
Напишите функцию, которая выполняет блок декодера upsampling
(например, с помощью ConvTranspose) и объединяет выходной тензор с
соответствующим тензором из encoder с помощью concatenation.
Протестируйте функцию на нескольких примерах, убедившись, что размеры
тензоров соответствуют ожиданиям.

Блок upsampling состоит из:
- операции апсемплинг + свертка, которую можно задать с помощью
nn.ConvTranspose2d
- конкатенации с тензором из энкодера (skipping connection)
- двух последовательных сверток с ядром 3*3 и функцией активации
Relu. Padding = 0


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


In [2]:

class UNetUpBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetUpBlock, self).__init__()
        # nn.ConvTranspose2d - транспонированная свертка, используется для увеличения пространственных размеров (upsampling).
        # Параметры:
        # in_channels - количество входных каналов,
        # out_channels - количество выходных каналов,
        # kernel_size=2 - размер ядра свертки 2x2,
        # stride=2 - шаг 2 для удвоения размера входа (например, 64x64 -> 128x128).
        self.upconv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

        # nn.Conv2d - обычная 2D-свертка для извлечения признаков.
        # Параметры:
        # in_channels - количество входных каналов,
        # out_channels - количество выходных каналов,
        # kernel_size=3 - размер ядра 3x3,
        # padding=0 - без заполнения, поэтому размер выходного тензора уменьшается.
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=0)

        # nn.ReLU - функция активации ReLU (Rectified Linear Unit),
        # inplace=True означает, что операция делается "на месте" для экономии памяти.
        self.relu = nn.ReLU(inplace=True)

        # Второй сверточный слой с теми же параметрами, для более глубокого извлечения признаков.
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=0)

    def forward(self, x, skip_connection):
        # x - входной тензор (например, из предыдущего слоя U-Net, с уменьшенным пространственным размером).
        # skip_connection - тензор из соответствующего слоя вниз по U-Net, для объединения деталей.

        # Применяем транспонированную свертку для увеличения размера изображения (upsampling).
        x = self.upconv(x)

        # Объединяем (конкатенируем) по канальному измерению (dim=1)
        # upsampled x и skip_connection, чтобы сохранить детали из ранних слоев.
        x = torch.cat([x, skip_connection], dim=1)

        # Пропускаем результат через первый сверточный слой для извлечения признаков.
        x = self.conv1(x)

        # Применяем активацию ReLU для нелинейности.
        x = self.relu(x)

        # Пропускаем через второй сверточный слой для более детальной обработки.
        x = self.conv2(x)

        # Снова ReLU активация.
        x = self.relu(x)

        # Возвращаем обработанный тензор.
        return x


In [3]:

# Создаем объект блока UNetUpBlock
# Параметры:
# in_channels=256 — количество каналов входного тензора (input_tensor),
# out_channels=128 — количество каналов на выходе блока.
upblock = UNetUpBlock(256, 128)

# Создаем случайный входной тензор с размерностью:
# batch_size=1, каналы=256, высота=100, ширина=100
# torch.randn генерирует тензор с нормальным распределением.
input_tensor = torch.randn(1, 256, 100, 100)

# Создаем тензор для skip connection с размерностью:
# batch_size=1, каналы=128, высота=200, ширина=200
# Он должен иметь в 2 раза большую высоту и ширину, чтобы соответствовать upsampling слою.
skip_tensor = torch.randn(1, 128, 200, 200)

# Передаем тензоры в блок
output = upblock(input_tensor, skip_tensor)

# Выводим размерность выходного тензора
# Ожидается размерность: batch_size=1, каналы=128, высота и ширина немного уменьшатся из-за сверток с padding=0,
# то есть будет примерно (1, 128, 196, 196) или меньше (зависит от точного поведения сверток).
print(output.shape)


torch.Size([1, 128, 196, 196])
